In [ ]:
!wget https://data.mendeley.com/public-files/datasets/jxmfrvhpyz/files/283004ff-e529-4c3c-a1ee-4fb90024dc94/file_downloaded --output-document flower_photos.zip

In [ ]:
!unzip -q flower_photos.zip

In [ ]:
!ls -lrt


In [ ]:

!ls -alt flower_photos

In [ ]:
!ls -lrt

In [ ]:
!ls

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

In [ ]:
traindf=pd.read_csv('flower_photos/all_labels.csv',dtype=str)

In [ ]:
traindf

In [ ]:
# Create data generator for training and validation
data_root = 'flower_photos/flowers'
IMAGE_SIZE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
BATCH_SIZE = 32

In [ ]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                interpolation="bilinear")

In [ ]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

valid_generator=valid_datagen.flow_from_dataframe(
dataframe=traindf,
directory=data_root,
x_col="file_name",
y_col="label",
subset="validation",
seed=10,
shuffle=True,
class_mode="categorical",
**dataflow_kwargs)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator=train_datagen.flow_from_dataframe(
dataframe=traindf,
directory=data_root,
x_col="file_name",
y_col="label",
subset="training",
seed=10,
shuffle=True,
class_mode="categorical",
**dataflow_kwargs)

In [ ]:
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)

In [ ]:
labels_idx = (train_generator.class_indices)
idx_labels = dict((v,k) for k,v in labels_idx.items())

In [ ]:
idx_labels

In [ ]:
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

Find label index and order of classes.

In [ ]:
labels_idx = (train_generator.class_indices)
idx_labels = dict((v,k) for k,v in labels_idx.items())

In [ ]:
idx_labels

In [ ]:
NUM_CLASSES = len(idx_labels)

In [ ]:

image_batch, label_batch = next(iter(train_generator))

fig, axes = plt.subplots(8, 4, figsize=(20, 40))
axes = axes.flatten()
for img, lbl, ax in zip(image_batch, label_batch, axes):
    ax.imshow(img)
    label_ = np.argmax(lbl)
    label = idx_labels[label_]
    ax.set_title(label)
    ax.axis('off')
#plt.tight_layout(h_pad=0.01, w_pad=1)
plt.show()



In [ ]:
'''
# build a custom model
mdl = tf.keras.Sequential([
    #tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', 
                           kernel_initializer='glorot_uniform', padding='same', input_shape =IMAGE_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', 
                           kernel_initializer='glorot_uniform', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name = 'custom_class')
])
mdl.build([None, 224, 224, 3])
'''


In [ ]:
mdl = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/feature_vector/1", trainable=False), 
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name = 'custom_class')
])
mdl.build([None, 224, 224, 3])

In [ ]:
mdl.summary()

In [ ]:
mdl.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

In [ ]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
hist = mdl.fit(
    train_generator,
    epochs=13, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps).history